# Run the following cell before compiling PDF.

In [8]:
# Make a print‑ready PDF that places text in the center of each 0.38″ circle
# on the SPOT‑1000 (12×16) grid. If you give more than 192 labels, it makes
# a multi‑page PDF. Shorter lists are padded with blanks.
import math
from pathlib import Path

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyBboxPatch
from matplotlib.backends.backend_pdf import PdfPages
import re, unicodedata

def compute_margins(page_w, page_h, box_w, box_h, pitch_x, pitch_y, cols, rows):
    """
    calculate margins on page_w × page_h, with cols×rows circles of diameter, spaced by pitch_x/pitch_y

    return (left_right_margin, top_bottom_margin)
    """
    grid_w = (cols - 1) * pitch_x + cols * box_w
    grid_h = (rows - 1) * pitch_y + rows * box_h

    margin_x = (page_w - grid_w) / 2
    margin_y = (page_h - grid_h) / 2

    return margin_x, margin_y

# remove zero-width & BOMs: U+200B..U+200D, U+2060, U+FEFF
_ZW_RE   = re.compile(r'[\u200B-\u200D\u2060\uFEFF]')
# remove control chars (category "C"), but keep \n for line breaks
def _strip_controls_keep_newlines(s: str) -> str:
    return ''.join(ch for ch in s if (unicodedata.category(ch)[0] != 'C' or ch == '\n'))

def clean_multiline_text(s: str) -> str:
    if not s:
        return ""
    s = unicodedata.normalize("NFKC", s)          # normalize ligatures/widths
    s = s.replace('\r\n', '\n').replace('\r', '\n')
    s = _ZW_RE.sub('', s)                         # remove zero-width/BOM
    s = _strip_controls_keep_newlines(s)          # drop other control chars
    # strip empty lines created by cleanup
    lines = [ln.strip() for ln in s.split('\n')]
    lines = [ln for ln in lines if ln]            # drop blanks
    return '\n'.join(lines)

def draw_label(ax, cx, cy, label, fs, outline=True, style='spot'): # or style = 'box'
    if outline:
      if style == 'spot':
        ax.add_patch(Circle((cx, cy), R, fill=False, linewidth=0.5))
        ax.text(cx, cy, str(label), ha="center", va="center", fontsize=fs)
      if style == 'box':
        ax.add_patch(
            FancyBboxPatch(
                (cx, cy),  # bottom left
                BOX_W, BOX_H,
                boxstyle=f"round,pad=0.02,rounding_size={0.05}",
                fill=False,
                linewidth=0.5,
            )
        )
        ax.text(cx + BOX_W/2, cy + BOX_H/2, label, ha="center", va="center", fontsize=fs)
    else:
      if style == 'spot':
        ax.add_patch(Circle((cx, cy), R, fill=False, linewidth=0.))
        ax.text(cx, cy, str(label), ha="center", va="center", fontsize=fs)
      if style == 'box':
        ax.add_patch(
            FancyBboxPatch(
                (cx, cy),  # bottom left
                BOX_W, BOX_H,
                boxstyle=f"round,pad=0.02,rounding_size={0.05}",
                fill=False,
                linewidth=0.,
            )
        )
        ax.text(cx + BOX_W/2, cy + BOX_H/2, str(label), ha="center", va="center", fontsize=fs)

def make_multiline_labels_pdf(labels, out_pdf_path, fs, outline = True, style = 'spot'):
    """labels: list of labels where \n indicates lines. Shorter lists padded with blanks; longer → multipage."""
    pages = max(1, math.ceil(len(labels) / N_PER_PAGE))
    with PdfPages(out_pdf_path) as pdf:
        for p in range(pages):
            chunk = labels[p*N_PER_PAGE : (p+1)*N_PER_PAGE]
            # pad with empty labels
            if len(chunk) < N_PER_PAGE:
                chunk += [''] * (N_PER_PAGE - len(chunk))
            fig = plt.figure(figsize=(PAGE_W, PAGE_H))
            ax = fig.add_axes([0, 0, 1, 1])
            ax.set_xlim(0, PAGE_W); ax.set_ylim(0, PAGE_H); ax.set_aspect('equal'); ax.set_axis_off()
            k = 0
            for r in range(ROWS):
                cy = cy0 - r * (V_PITCH + BOX_H)
                for c in range(COLS):
                  cx = cx0 + c * (H_PITCH + BOX_W)
                  draw_label(ax, cx, cy, chunk[k], fs, outline = outline, style = style)
                  k += 1
            pdf.savefig(fig, dpi=300, facecolor="white")
            plt.close(fig)
    return out_pdf_path

def read_labels(df):
  labels = []
  for _, row in df.iterrows():
    for cell in row:
      if type(cell) == str and len(cell) >= 0:
        label = clean_multiline_text(cell)
      elif pd.isna(cell): # Explicitly handle NaN values
        label = ' '
      else: # Handle any other types
        label = ' '
      labels.append(label)
  return labels

# Customize and Generate your label pdf.
You can edit labels here: https://docs.google.com/spreadsheets/d/1tEpZKgPjgwI1y1Z9xxLpbl4gJVbPJ_FU3w06zp3QzuE/edit?usp=sharing

## Upload your excel label sheet to "Files" (menu on the left).
- Right click on the uploaded file, copy path and paste it below to define `path`.
- How you fill in your excel sheet will be exactly how they match the label sheet. Your excel should have matching column # and row # as the label sheet.

In [3]:
# Paste your file path here.
path = '/content/s002 VNS blood draw labels.xlsx'

#Spot Tags

In [12]:
import pandas as pd
label_df = pd.read_excel(path, header=None, sheet_name=0)
label_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,ENRL\nCSP-SF-002\nP800,ENRL\nCSP-SF-002\nP800,ENRL\nCSP-SF-002\nP800,ENRL\nCSP-SF-002\nEDTA,ENRL\nCSP-SF-002\nEDTA,ENRL\nCSP-SF-002\nEDTA,ENRL\nCSP-SF-002\nEDTA,ENRL\nCSP-SF-002\nEDTA,ENRL\nCSP-SF-002\nEDTA,ENRL\nCSP-SF-002\nNull,ENRL\nCSP-SF-002\nNull,ENRL\nCSP-SF-002\nNull
1,ENRL\nCSP-SF-002\nLPS,ENRL\nCSP-SF-002\nLPS,ENRL\nCSP-SF-002\nLPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENRL\nCSP-SF-002\nPBMC,ENRL\nCSP-SF-002\nPBMC,ENRL\nCSP-SF-002\nPBMC,ENRL\nCSP-SF-002\nPBMC,ENRL\nCSP-SF-002\nPellet,ENRL\nCSP-SF-002\nPellet,ENRL\nCSP-SF-002\nPellet,ENRL\nCSP-SF-002\nPellet,ENRL\nCSP-SF-002\nSH,ENRL\nCSP-SF-002\nSH,ENRL\nCSP-SF-002\nSH,ENRL\nCSP-SF-002\nSH
3,ENRL\nCSP-SF-002\nNull,ENRL\nCSP-SF-002\nLPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chrl\nCSP-SF-002\nP800,Chrl\nCSP-SF-002\nP800,Chrl\nCSP-SF-002\nP800,Chrl\nCSP-SF-002\nEDTA,Chrl\nCSP-SF-002\nEDTA,Chrl\nCSP-SF-002\nEDTA,Chrl\nCSP-SF-002\nEDTA,Chrl\nCSP-SF-002\nEDTA,Chrl\nCSP-SF-002\nEDTA,Chrl\nCSP-SF-002\nNull,Chrl\nCSP-SF-002\nNull,Chrl\nCSP-SF-002\nNull
5,Chrl\nCSP-SF-001\nLPS,Chrl\nCSP-SF-002\nLPS,Chrl\nCSP-SF-002\nLPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Chrl\nCSP-SF-001\nPBMC,Chrl\nCSP-SF-002\nPBMC,Chrl\nCSP-SF-002\nPBMC,Chrl\nCSP-SF-002\nPBMC,Chrl\nCSP-SF-002\nPellet,Chrl\nCSP-SF-002\nPellet,Chrl\nCSP-SF-002\nPellet,Chrl\nCSP-SF-002\nPellet,Chrl\nCSP-SF-002\nSH,Chrl\nCSP-SF-002\nSH,Chrl\nCSP-SF-002\nSH,Chrl\nCSP-SF-002\nSH
7,Chrl\nCSP-SF-001\nNull,Chrl\nCSP-SF-002\nLPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Customize parameters below, then hit run.
- Download the generated pdf from Files.

In [14]:
# --- SPOT TAGS ---

# --- Sheet spec (inches) ---
PAGE_W, PAGE_H = 8.5, 11.0
BOX_W, BOX_H = 3/8, 3/8
H_PITCH, V_PITCH = (0.63-BOX_W), (0.63-BOX_H)
COLS, ROWS = 12, 16
N_PER_PAGE = COLS * ROWS

LEFT_MARGIN, TOP_MARGIN = compute_margins(PAGE_W, PAGE_H, BOX_W, BOX_H, H_PITCH, V_PITCH, COLS, ROWS)

R = BOX_W / 2
cx0 = LEFT_MARGIN + R

# Matplotlib's origin is bottom-left; our TOP margin measured from top edge:
cy0 = PAGE_H - (TOP_MARGIN + R)

FONT_SIZE = 4.5

labels =  read_labels(label_df)
out_pdf = path.split('/')[-1].replace('.xlsx', '_spot_cryotube_tags.pdf')
make_multiline_labels_pdf(labels, out_pdf, fs=FONT_SIZE, outline = False, style = 'spot') # outline = False to disable tag outlining
out_pdf

's002 VNS blood draw labels_spot_cryotube_tags.pdf'

# Rectangle Tags

In [4]:
import pandas as pd
label_df = pd.read_excel(path, header=None, sheet_name=1)
label_df

,0,1,2,3,4,5
0,CSP-SF-002 \nDOB: 2/26/1986\nMRN: ﻿﻿60124245\n...,CSP-SF-002 \nDOB: 2/26/1986\nMRN: ﻿﻿60124245\n...,CSP-SF-002 \nDOB: 2/26/1986\nMRN: ﻿﻿60124245\n...,CSP-SF-002 \nDOB: 2/26/1986\nMRN: ﻿﻿60124245\n...,CSP-SF-002 \nDOB: 2/26/1986\nMRN: ﻿﻿60124245\n...,CSP-SF-002 \nDOB: 2/26/1986\nMRN: ﻿﻿60124245\n...


In [10]:
# --- RECTANGULAR TAGS ---

# --- Sheet spec (inches) ---
PAGE_W, PAGE_H = 8.5, 11.0
BOX_W, BOX_H = 0.94, 0.47
H_PITCH, V_PITCH = 0.126, 0.16
COLS, ROWS = 7, 17
N_PER_PAGE = COLS * ROWS

LEFT_MARGIN, TOP_MARGIN = compute_margins(PAGE_W, PAGE_H, BOX_W, BOX_H, H_PITCH, V_PITCH, COLS, ROWS)
LEFT_MARGIN = 0.56
TOP_MARGIN = 0.24

cx0 = LEFT_MARGIN

# Matplotlib's origin is bottom-left; our TOP margin measured from top edge:
cy0 = PAGE_H - TOP_MARGIN - BOX_H
FONT_SIZE = 5.3

labels =  read_labels(label_df)
out_pdf = path.split('/')[-1].replace('.xlsx', 'LCRY2380_box_tags.pdf')
make_multiline_labels_pdf(labels, out_pdf, fs=FONT_SIZE, outline = False, style = 'box')
out_pdf

's002 VNS blood draw labelsLCRY2380_box_tags.pdf'